# Analog p-bits

Finding p-bit designs is hard to scale, general ideas fall into one of three camps:

1. Sin: Digital Simulation
2. Budget: Analog Amplifier Mayhem
3. Smart: Exotic Novel Device Physics for big boys

Mostly because they don't exist yet and people wouldn't let me use them if they did, I am stuck in the first two camps. In this notebook, I'll obviously focus on option number two in particular.

In analog p-bits, we seek to match an analog variation of the description of p-bits that have been given to us in probabilistic computing literature: stochastic neurons with sigmoid activation. To do this, we have to have electronic stochastic processes that we can take advantage of to emulate the desired behavior. Generally speaking, there's only two noise sources that we can meaningfully leverage in CMOS:

1. Nyquist-Johnson or Thermal Noise
2. Shot or Zener-Breakdown Noise

The latter is a Poisson process that is difficult to control, requires special junctions which I don't believe are specifically designed for in the SKY130 PDK. Both require powerful amplifiers to be detected, but we can only really use the former. But putting a big amplifier in the middle of every p-bit is clearly a non-starter. 

### Ring-Oscillator Based Design

Ring oscillators are just 3 inverters connected end to end and then looped back, hence: RING oscillator. After being powered up these will ring out forever. So this is a cool toy, but how do I make it stochastic?

**Random Sampling Times**: whilst the ring oscillator is doing its thing, we use it to change the value of a flip-flop with a clock which is generated with real thermal noise. We can arrange these ring oscillators in an array and *carefully* isolate and detune them so they each have a unique random period, all of which will be sampled independently by unified random sampling signal (meaning less amps for us: yay)

This is a very small circuit, but it is a constant 50% duty cycle, flipping on and off as quickly as charges move through the inverters. For there to be a p-bit, we must be able to control the duty cycle of a ring oscillator, and it turns out, thanks to our Vietnamese friends in Japan, Minh-Hai Nguyen and  Cong-Kha Pham, we can!

In their paper, A Wide Frequency Range and Adjustable Duty Cycle CMOS Ring Voltage Controlled Oscillator, we learn that if we seperate the inverters with transmission gates, we can control the duty-cycle indepedently of the frequency (although, we can also control the frequency!).

In [ ]:
# hdl21
import hdl21 as h
from hdl21.prefix import μ, n
from hdl21.sim import Sim

# vlsirtools
import vlsirtools.spice as vsp

# Import the sky130 library
import sky130
from sky130 import modules as s

@h.module
class transmission_gate:

    # IO
    input = h.Input()
    output = h.Output()
    ctrl = h.Signal()

    # Instantiate the transmission gate
    tg = s.PMOS_1p8V_STD()(d=input, g=ctrl, s=output, b=input)
    bg = s.PMOS_1p8V_STD()(d=input, g=ctrl, s=output, b=output)

# First create the ring oscillator
@h.module
class ring_oscillator:

    # IO
    input = h.Input()
    output = h.Output()

    # Signals
    n1, n2, n3, n4 = h.Signal(4)
    gnd, nb, pb, pwr, ctrl = h.Signal(5)

    # Necessary parameters
    lp = sky130.Sky130LogicParams()

    # Instantiate the inverters
    I1 = s.sky130_fd_sc_hd__inv_1(lp)(A=input, VGND=gnd, VNB=nb, VPB=pb, VPWR=pwr, Y=n1)
    TG1 = transmission_gate()(input=n1, output=n2, ctrl=ctrl)
    I2 = s.sky130_fd_sc_hd__inv_1(lp)(Y=n2, VGND=gnd, VNB=nb, VPB=pb, VPWR=pwr, Z=n3)
    TG2 = transmission_gate()(input=n3, output=n4, ctrl=ctrl)
    I3 = s.sky130_fd_sc_hd__inv_1(lp)(
        Z=n3, VGND=gnd, VNB=nb, VPB=pb, VPWR=pwr, A=n4
    )
    TG3 = transmission_gate()(input=n4, output=n1, ctrl=ctrl)

    output = n1

@h.paramclass
class detuner_params:
    lr: h.Prefixed = 0.18 * μ
    wr: h.Prefixed = 0.18 * μ
    lc: h.Prefixed = 0.18 * μ
    wc: h.Prefixed = 0.18 * μ


# Now create the detuner
@h.generator
def detuner_gen(params: detuner_params) -> h.Module:
    
    @h.module
    class detuner:

        # IO
        input = h.Input()
        output = h.Output()

        # Signals
        n1, gnd = h.Signal(2)

        rp = sky130.Sky130GenResParams(l=params.lr, w=params.wr)
        cp = sky130.Sky130VarParams(l=params.lc, w=params.wc)

        # Instantiate the resistor and capacitor
        R1 = s.GEN_PO(rp)(A=input, B=n1)
        C1 = s.VAR_LVT(cp)(p=n1, n=gnd, b=gnd)

        output = n1

    return detuner

@h.sim.sim
class pbit_sim():

    @h.module
    class Tb():

        # Instantiate the ring oscillator
        RO = ring_oscillator()

        # Instantiate the detuner
        dp = detuner_params(lr=0.18*μ, wr=0.18*μ, lc=0.18*μ, wc=0.18*μ)
        detuner = detuner_gen(dp)()

        # Connect the ring oscillator and the detuner
        detuner.input = RO.output
        RO.input = detuner.output

        VSS = h.Port() # Ground is a single port

        # Connect the power signals to the ring oscillator
        VDD = h.DcVoltageSource(p=RO.pwr, n=VSS, v=1.8)
        RO.ppb = VDD
        RO.pnb = VSS
        RO.gnd = VSS

        # Ground the detuner
        detuner.gnd = VSS

        # Connect a PulseVoltageSource to the input of the ring oscillator
        Vpulse = h.PulseVoltageSource(p=RO.input, n=VSS, delay=0,v1=1.8,width=1*n)

    # Run a transient simulation
    opts = vsp.SimOptions(
    simulator=vsp.SupportedSimulators.NGSPICE,
    fmt=vsp.ResultFormat.SIM_DATA,
    rundir="./scratch",
    )

    rv = Sim.run(opts)